In [5]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from dateutil.parser import parse
import datetime
os.getcwd()

'/home/Hyperspectral_predict/PSC'

# **Dataset preprocessing**

In [6]:
def data_set (data) :
    day_df = pd.DataFrame(columns = ["Year", "Month", "Day", "Hour", "Start Min", "End Min"] + list(data.columns))

    for i in tqdm(range(data.shape[0])) : 
        obs = data.iloc[[i], :]                                         # obs : 데이터 하나
        time = parse(obs["timestamp"].values[0])                        # obs의 time 정보
        time = time + datetime.timedelta(hours = 9)

        # start = 0이면 해당 날짜의 데이터 수집 시작 알림
        if (time.hour == 0) & (time.minute < 10) :
            print("{}년 {}월 {}일 데이터 수집 시작".format(time.year, time.month, time.day))
        
        # obs의 관측 time(hour)이 하루가 지나면 데이터 수집 종료
        elif (time.hour == 23) & (time.minute > 50) :
            print("{}년 {}월 {}일 데이터 수집 종료".format(time.year, time.month, time.day))

        # obs의 관측 time(hour)이 0~23이면 데이터 수집
        if time.hour in range(0, 24) :
            Y, M, D, H, Min = time.year, time.month, time.day, time.hour, time.minute

            if time.minute in range(0, 15+1) :
                inform = pd.DataFrame([Y, M, D, H, 0, 15] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(15, 30+1) :
                inform = pd.DataFrame([Y, M, D, H, 15, 30] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(30, 45+1) :
                inform = pd.DataFrame([Y, M, D, H, 30, 45] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(45, 60+1) :
                inform = pd.DataFrame([Y, M, D, H, 45, 0] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)
        

    day_df = day_df.drop(columns = ["timestamp"])
    complete_data = day_df.groupby(["Year", "Month", "Day", "Hour", "Start Min", "End Min"]).sum()
    
    return complete_data

In [7]:
def data_set2 (data) :
    day_df = pd.DataFrame(columns = ["Year", "Month", "Day", "Hour", "Min"] + list(data.columns))

    for i in tqdm(range(data.shape[0])) : 
        obs = data.iloc[[i], :]                                         # obs : 데이터 하나
        time = parse(obs["timestamp"].values[0])                        # obs의 time 정보
        time = time + datetime.timedelta(hours = 9)

        Y, M, D, H, Min = time.year, time.month, time.day, time.hour, time.minute
        
        if time.minute in range(0, 15) :
            inform = pd.DataFrame([Y, M, D, H, 15] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(15, 30) :
            inform = pd.DataFrame([Y, M, D, H, 30] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(30, 45) :
            inform = pd.DataFrame([Y, M, D, H, 45] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(45, 60) :
            time2 = time + datetime.timedelta(minutes = 15)
            Y_, M_, D_, H_= time2.year, time2.month, time2.day, time2.hour
            inform = pd.DataFrame([Y_, M_, D_, H_, 0] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)


    day_df = day_df.drop(columns = ["timestamp"]).reset_index(drop = True)
        
    return day_df

In [8]:
csv_name = "yangpyeong 220201-221005-Copy1.csv"
data = pd.read_csv("../DAT/" +csv_name)
print(data.shape)
data.head()

(38339, 569)


,timestamp,waterTemperature,conductivity,do,ph,turbidity,chlorophyll,phycocyanin,waterDepth,airTemperature,...,reflectance.545,reflectance.546,reflectance.547,reflectance.548,reflectance.549,reflectance.550,chla,tsm,kd,cpc
0,20220201T001248,2.68,0.31,14.02,8.210,2.28,4.05,0.07,0.59,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220201T001250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220201T002748,2.74,0.31,14.03,8.220,2.32,3.88,0.06,0.60,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220201T002750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20220201T004248,2.75,0.31,13.99,8.214,2.67,3.45,0.08,0.61,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
water = data[["timestamp", 'chlorophyll', 'conductivity', 'do', 'ph', 'phycocyanin', 'turbidity', 'waterDepth', 'waterTemperature']].dropna().reset_index(drop = True)
weather = data[["timestamp", 'airTemperature', 'humidity', 'insolation', 'windDirection', 'windSpeed']].dropna().reset_index(drop = True)
hs = data[["timestamp"] +["chla"]+ list(data.filter(regex = 'reflectance').columns)].dropna().reset_index(drop = True)

print(water.shape, weather.shape, hs.shape)

(16172, 9) (16171, 6) (5769, 553)


In [1]:
# hs.head()

In [12]:
new_water = data_set2(water)
new_weather = data_set2(weather)
new_hs = data_set2(hs)

  0%|          | 0/16172 [00:00<?, ?it/s]

  0%|          | 0/16171 [00:00<?, ?it/s]

  0%|          | 0/5769 [00:00<?, ?it/s]

In [13]:
display(new_water.head(3))
display(new_weather.head(3))
display(new_hs.head(3))

,Year,Month,Day,Hour,Min,chlorophyll,conductivity,do,ph,phycocyanin,turbidity,waterDepth,waterTemperature
0,2022,2,1,9,15,4.05,0.31,14.02,8.21,0.07,2.28,0.59,2.68
1,2022,2,1,9,30,3.88,0.31,14.03,8.22,0.06,2.32,0.6,2.74
2,2022,2,1,9,45,3.45,0.31,13.99,8.214,0.08,2.67,0.61,2.75


,Year,Month,Day,Hour,Min,airTemperature,humidity,insolation,windDirection,windSpeed
0,2022,2,1,9,15,-1.1,77.5,168.9,22.8,0.9
1,2022,2,1,9,30,-1.1,74.6,164.1,22.8,5.4
2,2022,2,1,9,45,-1.2,77,175.9,117.5,0.4


,Year,Month,Day,Hour,Min,chla,reflectance.000,reflectance.001,reflectance.002,reflectance.003,...,reflectance.541,reflectance.542,reflectance.543,reflectance.544,reflectance.545,reflectance.546,reflectance.547,reflectance.548,reflectance.549,reflectance.550
0,2022,3,4,11,45,10.4,0.00486176,0.00483305,0.00477715,0.00471701,...,0.00085363,0.00083233,0.00080247,0.00076564,0.00075941,0.00072744,0.00073683,0.00074251,0.00075114,0.00073995
1,2022,3,4,12,0,10.4,0.00484752,0.00479738,0.00475028,0.0047138,...,0.00101725,0.00100019,0.00099312,0.00094897,0.00090831,0.00092162,0.00088647,0.00086212,0.00085729,0.00087079
2,2022,3,4,12,15,10.7,0.00496335,0.00496847,0.00492096,0.00486571,...,0.00087999,0.00085441,0.00084285,0.00081966,0.00080705,0.00079802,0.00078827,0.0007723,0.00074203,0.00072364


In [14]:
new_data = pd.merge(left = new_water, right = new_weather, how = 'outer', on = ["Year", "Month", "Day", "Hour", "Min"], sort = True)
new_data = pd.merge(left = new_data, right = new_hs, how = 'outer', on = ["Year", "Month", "Day", "Hour", "Min"], sort = True)  
new_data = new_data.astype({"Year" : "int", "Month" : "int", "Day" : "int", "Hour" : "int", "Min" : "int"})
new_data.head(3)

,Year,Month,Day,Hour,Min,chlorophyll,conductivity,do,ph,phycocyanin,...,reflectance.541,reflectance.542,reflectance.543,reflectance.544,reflectance.545,reflectance.546,reflectance.547,reflectance.548,reflectance.549,reflectance.550
0,2022,2,1,9,15,4.05,0.31,14.02,8.21,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,2,1,9,30,3.88,0.31,14.03,8.22,0.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,2,1,9,45,3.45,0.31,13.99,8.214,0.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# new_data.to_csv("../DAT/complete_data(yp).csv", index = False)

In [16]:
new_data.to_csv("../DAT/complete_data(yang).csv", index =False )